In [1]:
import numpy as np
import os
from scipy.integrate import quad
from scipy.stats import multivariate_normal as mvn
from scipy.stats import norm
from scipy.optimize import minimize
from import_file import import_file
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from ambit_stochastics.trawl import trawl

Let $(l_1,\ldots,l_n) = {X_s,\ldots,X_{s\cdot  n}}$ be the values of a trawl process $X$ observed on a grid with spacing $s$, which is known. We restrict our attention to exponential and gamma envelopes and to Gaussian Levy seeds $L^{'}$ and we try to infer the parameters of the envelope, as well as the mean $\mu_L $and variace $V_L$ of $L^{'}$. 





Assume the trawl process has one of the following three envelope functions $\phi \colon (-\infty,0] \to \mathbb{R}_{\ge 0}$:

$$
\phi_{\lambda}(t) = \begin{cases} 
\lambda e^{\lambda t} &\text{ if } t \le 0 \\ 
0 &\text{ if } t > 0
\end{cases} $$

for some $\lambda > 0 $ or 

$$
\phi_{H,\delta}(t) = \begin{cases} 
\frac{H}{\delta} \left(1-\frac{s}{\delta}\right)^{-(H+1)} &\text{ if } t \le 0 \\ 
0 &\text{ if } t > 0
\end{cases} 
$$
for some $H,\delta > 0$ or 


The above functions are normalised such that the Lebesgue measure of the ambit set is $1$. 


Let $\theta_L = (\mu_L,V_L), \theta_{\text{env}} = \lambda  $ or $\theta_{\text{env}} =(H,\delta)$ or $\theta_{\text{env}}=(\delta,\gamma)$. depending on which envelope we use and let $\theta = (\theta_L,\theta_{\text{env}})$. Consider the composite log-likelihood with lag-indices $0 = j_1 < \ldots < j_m = k$ given by 
\begin{equation}
l_J(\theta)= \sum_{i=1}^{i=n-k} p(x_i; \theta)
\end{equation}
where $J = (j_1,\ldots,j_k)$ and $x_i = \left(l_i=l_{i+j_0},l_{i+j_1},\ldots,l_{i+j_{m-1}}, l_{i+j_m} =l_{i+k}\right).$ In this noteobok, we will investigate whether the composite likelihood approach performs better than the method of moments approach and study the effect the choice $k$ has on the inference. Considering more than $k=1$ can be prohibitively expensive for non-Gaussian Levy seeds.



#### Composite likelihood 

Let 
\begin{equation}
l_k(\theta) = - \frac{m(n-k)}{2} \log{(2 \pi)}- \frac{n-k}{2} \log{(\det{\Sigma(\theta_{\text{env}})})} - \frac{1}{2}\sum_{i=1}^{n-k}\left(x_i-\mu\right)^T\Sigma^{-1}\left(x_i-\mu\right) \label{l}
\end{equation}

where $\mu = [\mu_L,\ldots,\mu_L]$ and $\Sigma = V_L \cdot \tilde{\Sigma(\theta_{\text{env}})}.$ Then




\begin{equation}\label{dl/dmu}
\frac{\partial l}{\partial \mu}  =  \sum_{i=1}^{n-k}(x_i-\mu)^T \Sigma^{-1}  \text{ hence by chain rule } \frac{\partial l}{\partial \mu_L} =  \frac{\partial l}{\partial \mu} \cdot [\mu_L,\ldots,\mu_L
]  =  \frac{1}{V_L}\left[Sum\left(\left(\sum_{i=1}^{n-k} x_i^T\right) \tilde{\Sigma}^{-1}\right) - (n-k) \mu_L Sum\left(\tilde{\Sigma}^{-1}\right)\right] 
\end{equation}

\begin{equation} \label{dl\dV}
\frac{\partial l}{\partial V_L} = -\frac{1}{2} \frac{\partial}{\partial V_L} \left[\log{\left(\det{\left(V_L \cdot \tilde{\Sigma}\right)}\right)}+  (x-\mu)^T {(V_L \cdot \tilde{\Sigma})}^{-1}(x-\mu)\right] = -\frac{m(n-k)}{2 V_L} + \sum_{i=1}^{n-k}\frac{1}{2V_L^2}(x_i-\mu)^T {\tilde{\Sigma}}^{-1}(x_i-\mu)
\end{equation}

$$\frac{\partial l}{\partial \theta_{\text{env}}} = -\frac{1}{2} \sum_{i=1}^{n-k} \left[ \frac{\frac{\partial}{\partial \theta_\text{env}}{\det\left(\tilde{\Sigma}\left(\theta_{\text{env}}\right)\right)}}{det \left(\tilde{\Sigma}\left(\theta_{\text{env}}\right)\right)} + \frac{1}{V_L} \left(x-\mu\right)^T \frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma}^{-1}\left(\theta_{\text{env}}\right)\left(x-\mu\right) 
\right]$$

By [Jacobi's formula](https://en.wikipedia.org/wiki/Jacobi%27s_formula)
  $$ \frac{\partial}{\partial \theta_\text{env}}{\det\left(\tilde{\Sigma}\left(\theta_{\text{env}}\right)\right)} = \det\left(\tilde{\Sigma}\left(\theta_\text{env}\right)\right)Tr\left(\tilde{\Sigma}^{-1}\frac{\partial }{\partial \theta_{\text{env}}} \tilde{\Sigma}\right)$$
  hence 
  $$ \frac{\partial}{\partial \theta_\text{env}}{\log \det\left(\tilde{\Sigma}\left(\theta_{\text{env}}\right)\right)}  = Tr\left(\tilde{\Sigma}^{-1}\frac{\partial }{\partial \theta_{\text{env}}} \tilde{\Sigma}\right)
  $$ 
  and further by 
  $$ 0 = \frac{\partial}{\partial \theta_\text{env}} \left(\tilde{\Sigma} \tilde{\Sigma}^{-1}\right) = \tilde{\Sigma}\  \frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma}^{-1} + \left( \frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma}\right) \tilde{\Sigma}^{-1} $$
  we have
  $$ \frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma}^{-1} = -\tilde{\Sigma}^{-1} \left(\frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma} \right) \ \tilde{\Sigma}^{-1} 
  $$ 

hence $$\frac{\partial l}{\partial \theta_{\text{env}}} = -\frac{1}{2}\left[(n-k) Tr\left(\tilde{\Sigma}^{-1}\frac{\partial }{\partial \theta_{\text{env}}} \tilde{\Sigma}\right) - \frac{1}{V_L} \sum_{i=1}^{n-k} \left(x_i-\mu\right)^T \tilde{\Sigma}^{-1} \left(\frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma} \right) \ \tilde{\Sigma}^{-1}\left(x_i-\mu\right) 
\right]$$


#### Gradients
Thus we have the following expressions for the gradients:
\begin{align}
\frac{\partial l}{\partial \mu} &= \frac{1}{V_L}\left[Sum\left(\left(\sum_{i=1}^{n-k} x_i^T\right) \tilde{\Sigma}^{-1}\right) - (n-k) \mu_L Sum\left(\tilde{\Sigma}^{-1}\right)\right]  \\
\frac{\partial l}{\partial V_L}  &= -\frac{m(n-k)}{2 V_L} + \sum_{i=1}^{n-k}\frac{1}{2V_L^2}(x_i-\mu)^T {\tilde{\Sigma}}^{-1}(x_i-\mu)\\
\frac{\partial l}{\partial \theta_{\text{env}}} &= -\frac{1}{2}\left[(n-k) Tr\left(\tilde{\Sigma}^{-1}\frac{\partial }{\partial \theta_{\text{env}}} \tilde{\Sigma}\right) - \frac{1}{V_L} \sum_{i=1}^{n-k} \left(x_i-\mu\right)^T \tilde{\Sigma}^{-1} \left(\frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma} \right) \ \tilde{\Sigma}^{-1}\left(x_i-\mu\right) 
\right]
\end{align}

We can proceed by BFGS or we can notice that for a given $\theta_\text{env},$ the optimal $\mu(\theta_\text{env})$ and $V_L(\theta_\text{env})$ are given by

\begin{align}
\mu_L(\theta_\text{env}) &=    \frac{Sum\left(\left(\sum_{i=1}^{n-k} x_i^T\right) \tilde{\Sigma}^{-1}(\theta_\text{env})\right)}{(n-k)Sum\left(\tilde{\Sigma}^{-1}(\theta_\text{env})\right)}  \\
V_L(\theta_\text{env}) &= \frac{\sum_{i=1}^{n-k}\left(x_i-[\mu_L(\theta_\text{env}),\ldots,\mu_L(\theta_\text{env})]\right)^T {\tilde{\Sigma}}^{-1}(\theta_\text{env})\left(x_i-[\mu_L(\theta_\text{env}),\ldots,\mu_L(\theta_\text{env})]\right)}{m(n-k)}
\end{align}

By substituting this back into the likelihood formula we have
\begin{equation}
\tilde{l}_k\left(\theta_\text{env}\right)  = l_k\left(\theta_\text{env},\mu_L(\theta_\text{env}\right),V_L(\theta_\text{env})) = - \frac{m(n-k)}{2} \left(\log{(2 \pi)}+1\right)- \frac{n-k}{2} \left[\log{(\det{\tilde{\Sigma}(\theta_{\text{env}})})} + m \log V_L(\theta_\text{env}) \right] 
\end{equation}
and thus
\begin{align}
\frac{\partial \tilde{l}}{\partial \theta_{\text{env}}} = &-\frac{n-k}{2}   \left[Tr\left(\tilde{\Sigma}^{-1}\frac{\partial }{\partial \theta_{\text{env}}} \tilde{\Sigma}\left(\theta_{\text{env}} \right)\right) \right] \\
&+ \frac{1}{2 V_L(\theta_\text{env})} \left(\sum_{i=1}^{n-k}\left(x_i-[\mu_L(\theta_\text{env}),\ldots,\mu_L(\theta_\text{env})]\right)^T \tilde{\Sigma}^{-1}\left(\theta_{\text{env}}\right) \left(\frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma} \right) \ \tilde{\Sigma}^{-1}\left(\theta_{\text{env}}\right)  \left(x_i-[\mu_L(\theta_\text{env}),\ldots,\mu_L(\theta_\text{env})]\right) \right)
\end{align}

In [2]:
def corr(s,J,envelope,envelope_params):
    """return overlap area (i.e. correlation) at lags. these are equivalent because the area of the 
    ambit set is normalised to 1. lags is a SORTED numpy array"""
    
    assert envelope in ['gamma','exponential']
    assert isinstance(J,np.ndarray)
    
    if envelope == 'exponential':
        u = envelope_params[0]
        areas = np.exp(- u * s * J )
        
    elif envelope == 'gamma':
        H,delta = envelope_params
        areas = (1+J*s/delta)**(-H)
        
    return list(areas)

In [3]:
#corr(0.1,np.array((0,1,2)),'exponential',(10,))
#import numpy as np
corr(1,np.array((0,1,2,3)),'gamma',(1.5,0.75))

[1.0, 0.2805658588748474, 0.14242717305466188, 0.08944271909999159]

In [4]:
def gradients_corr(s,J,envelope,envelope_params):
    """return gradients of the correlation function at lags  with respect to the envelope _params"""
    

    assert envelope in ['gamma','exponential']
    
    if envelope == 'exponential':
        u = envelope_params[0]
        gradients_areas = -s * J * np.exp(- u * s * J )
        #print(J)
        #print(gradients_areas)
        return list(gradients_areas)
        
    elif envelope == 'gamma':
        H,delta = envelope_params
        
        d_areas_d_H     = - np.log(1+J*s/delta) * (1+J*s/delta)**(-H)
        d_areas_d_delta =  H * s * J* (1+J*s/delta)**(-H-1)/delta**2     
        
        return list(d_areas_d_H),list(d_areas_d_delta)

In [5]:
def Sigma_tilde_func(s,J,envelope,envelope_params):
    """inputs a vector = [corr(0),corr(s),...,corr((k-1)*s)] and outputs
    Sigma_tilde_ij = overlap area at lag |i-j| = corr(|i-j|*s)
    """
    m = len(J)
    overlap_area_vector = corr(s,J,envelope,envelope_params)
    Sigma_tilde = [overlap_area_vector[1:i+1][::-1] + overlap_area_vector[:m-i] for i in range(m)]
    Sigma_tilde = np.array(Sigma_tilde)
    return Sigma_tilde

In [6]:
#np.linalg.det(Sigma_tilde_func(0.35, np.array((0,1,2,3,5)),'exponential',(1.0,)))
Sigma_tilde_func(0.35, np.array((0,1)),'exponential',(1.0,))

array([[1.        , 0.70468809],
       [0.70468809, 1.        ]])

In [7]:
def gradients_Sigma_tilde_func(s,J,envelope,envelope_params):
    """inputs a vector = [corr(0),corr(s),...,corr((k-1)*s)] and outputs the gradients of
    Sigma_tilde_ij = overlap area at lag |i-j| = corr(|i-j|*s) with respect to the parameters of the envelope: TO UPDATE
    """
    assert envelope in ['gamma','exponential']
    m = len(J)
    #k = max(J)
    
    if envelope == 'exponential':
        d_overlap_area_vector_d_u = gradients_corr(s,J,envelope,envelope_params)
        d_Sigma_tilde_d_u = [d_overlap_area_vector_d_u[1:i+1][::-1] + d_overlap_area_vector_d_u[:m-i] for i in range(m)]
        d_Sigma_tilde_d_u = np.array(d_Sigma_tilde_d_u)
        
        return d_Sigma_tilde_d_u
    
    if envelope == 'gamma':
        d_overlap_area_vector_d_H, d_overlap_area_vector_d_delta = gradients_corr(s,J,envelope,envelope_params)
        d_Sigma_tilde_d_H = [d_overlap_area_vector_d_H[1:i+1][::-1] + d_overlap_area_vector_d_H[:m-i] for i in range(m)]
        d_Sigma_tilde_d_delta = [d_overlap_area_vector_d_delta[1:i+1][::-1] + d_overlap_area_vector_d_delta[:m-i] for i in range(m)]

        return np.array(d_Sigma_tilde_d_H),np.array(d_Sigma_tilde_d_delta)

In [8]:
#gradients_Sigma_tilde_func(tau,J,envelope,(1.25,))

In [9]:
def composite_likelihoods(joints,J,mu_L,V_L,Sigma_tilde):
    """covariance matrix = Sigma_tilde * V0
    Sigma_tilde has to be recomputed if any of the gaussian part params 
    
    mu0 and V0 
    
    change"""
    m = len(J) 
    result = np.array([mvn.logpdf(joint, mean = mu_L * np.ones(m) , cov = V_L * Sigma_tilde) #there was a divided by 2 here, i think it was an error
     for joint in joints])
    return result


\begin{align}
\mu_L(\theta_\text{env}) &=    \frac{Sum\left(\left(\sum_{i=1}^{n-k} x_i^T\right) \tilde{\Sigma}^{-1}(\theta_\text{env})\right)}{(n-k)Sum\left(\tilde{\Sigma}^{-1}(\theta_\text{env})\right)}  \\
V_L(\theta_\text{env}) &= \frac{\sum_{i=1}^{n-k}\left(x_i-[\mu_L(\theta_\text{env}),\ldots,\mu_L(\theta_\text{env})]\right)^T {\tilde{\Sigma}}^{-1}(\theta_\text{env})\left(x_i-[\mu_L(\theta_\text{env}),\ldots,\mu_L(\theta_\text{env})]\right)}{m(n-k)}
\end{align}

\begin{equation}
\tilde{l}_k\left(\theta_\text{env}\right)  = l_k\left(\theta_\text{env},\mu_L(\theta_\text{env}\right),V_L(\theta_\text{env})) = - \frac{m(n-k)}{2} \left(\log{(2 \pi)}+1\right)- \frac{n-k}{2} \left[\log{(\det{\tilde{\Sigma}(\theta_{\text{env}})})} + m \log V_L(\theta_\text{env}) \right] 
\end{equation}

In [10]:
def l_hat(s,J,envelope,envelope_params,joints,n): 
    assert max(J) + len(joints) == n
    k = max(J)
    m = len(J)
    Sigma_tilde = Sigma_tilde_func(s,J,envelope,envelope_params)
    Sigma_tilde_inverse = np.linalg.pinv(Sigma_tilde)
    #print(Sigma_tilde_inverse)
    
    sum_x_i = np.sum(joints,axis=0)
    sum_Sigma_tilde_inverse = np.sum(Sigma_tilde_inverse)
    
    mu_L = np.sum(sum_x_i @ Sigma_tilde_inverse) / ((n-k)* sum_Sigma_tilde_inverse)
    V_L  = np.sum([(joint-mu_L) @ Sigma_tilde_inverse @(joint-mu_L) for joint in joints])/(m*(n-k))
    print(mu_L,V_L)
    #print(mu_L,V_L)
    #only works for 
   # if envelope == 'exponential':
    #    return -k*(n-k)*(np.log(2*np.pi)+1) /2 - (n-k)*((k-1)*np.log(1-np.exp(-envelope_params[0] * s * 2)) + k * np.log(V_L))/2
    
    #else:
    return -m*(n-k)*(np.log(2*np.pi)+1) /2 - (n-k)*(np.log(np.linalg.det(Sigma_tilde)) + m * np.log(V_L))/2

\begin{align}
\frac{\partial \tilde{l}}{\partial \theta_{\text{env}}} = &-\frac{n-k}{2}   \left[Tr\left(\tilde{\Sigma}^{-1}\frac{\partial }{\partial \theta_{\text{env}}} \tilde{\Sigma}\left(\theta_{\text{env}} \right)\right) \right] \\
&+ \frac{1}{2 V_L(\theta_\text{env})} \left(\sum_{i=1}^{n-k}\left(x_i-[\mu_L(\theta_\text{env}),\ldots,\mu_L(\theta_\text{env})]\right)^T \tilde{\Sigma}^{-1}\left(\theta_{\text{env}}\right) \left(\frac{\partial}{\partial \theta_\text{env}} \tilde{\Sigma} \right) \ \tilde{\Sigma}^{-1}\left(\theta_{\text{env}}\right)  \left(x_i-[\mu_L(\theta_\text{env}),\ldots,\mu_L(\theta_\text{env})]\right) \right)
\end{align}

In [11]:
def gradients_l_hat(s,J,envelope,envelope_params,joints,n):
    Sigma_tilde = Sigma_tilde_func(s,J,envelope,envelope_params)
    Sigma_tilde_inverse = np.linalg.pinv(Sigma_tilde)
    
    gradients_Sigma_tilde = gradients_Sigma_tilde_func(s,J,envelope,envelope_params)
    assert max(J) + len(joints) == n
    k = max(J)
    m = len(J)

    sum_x_i = np.sum(joints,axis=0)
    sum_Sigma_tilde_inverse = np.sum(Sigma_tilde_inverse)
    
    mu_L = np.sum(sum_x_i @ Sigma_tilde_inverse) / ((n-k)* sum_Sigma_tilde_inverse)
    V_L  = np.sum([(joint-mu_L) @ Sigma_tilde_inverse @(joint-mu_L) for joint in joints])/(m*(n-k))
    
    #if envelope == 'exponential':
    #        
    #    #term1 = -(n-k) * np.trace(Sigma_tilde_inverse @ gradients_Sigma_tilde)/2 
    #    term1 = -(n-k) * (k-1) * s * np.exp(-2 * envelope_params[0] * s) / (1-np.exp(-2*s*envelope_params[0]))
    #    term2 = np.sum([(joint-mu_L) @ Sigma_tilde_inverse @ gradients_Sigma_tilde @  Sigma_tilde_inverse @ (joint-mu_L) for joint in joints]) / (2 * V_L)
    #    #print(term1,term3)
    
    if envelope == 'exponential':
        term1 = -(n-k) * np.trace(Sigma_tilde_inverse @ gradients_Sigma_tilde)/2
        term2 = np.sum([(joint-mu_L) @ Sigma_tilde_inverse @ gradients_Sigma_tilde @  Sigma_tilde_inverse @ (joint-mu_L) for joint in joints]) / (2 * V_L)

    
    elif envelope == "gamma":

        term1 = np.array([-(n-k) * np.trace(Sigma_tilde_inverse @ i)/2 for i in gradients_Sigma_tilde])
        term2 = np.array([np.sum([(joint-mu_L) @ Sigma_tilde_inverse @ i @  Sigma_tilde_inverse @ (joint-mu_L) for joint in joints]) for i in gradients_Sigma_tilde]) / (2 * V_L)
                
    return term1 + term2

In [12]:
def optimise_l_hat(s,J,envelope,x0,joints,n):
    #x0 - initial envelope params
    #minimize minus the likelihood. 
    #
    fun = lambda x : -l_hat(s,J,envelope,x,joints,n)
    jac  = lambda x: -gradients_l_hat(s,J,envelope,x,joints,n)
    
    if envelope == 'gamma':
        bounds  = ((0.001,np.inf),(0.001,np.inf))
    elif envelope == 'exponential':
        bounds = ((0.001,np.inf),)
    
    result = minimize(fun = fun,jac= jac, x0=x0, method='L-BFGS-B',bounds=bounds)#, options = {"maxiter":200, "disp":True})
    return result

In [13]:
def mu_L_V_L(s,J,envelope,envelope_params,joints,n):
    Sigma_tilde = Sigma_tilde_func(s,J,envelope,envelope_params)
    Sigma_tilde_inverse = np.linalg.pinv(Sigma_tilde)
    
    #gradients_Sigma_tilde = gradients_Sigma_tilde_func(s,k,envelope,envelope_params)
    assert max(J) + len(joints) == n
    k = max(J)
    m = len(J)
    sum_x_i = np.sum(joints,axis=0)
    sum_Sigma_tilde_inverse = np.sum(Sigma_tilde_inverse)
    
    mu_L = np.sum(sum_x_i @ Sigma_tilde_inverse) / ((n-k)* sum_Sigma_tilde_inverse)
    V_L  = np.sum([(joint-mu_L) @ Sigma_tilde_inverse @(joint-mu_L) for joint in joints])/(m*(n-k))
    return mu_L,V_L

In [14]:
#(l_hat(tau,J,'exponential',[1.5],joints,nr_trawls) - l_hat(tau,J,'exponential',[1.499],joints,nr_trawls)) / 0.001
#gradients_l_hat(s,J,envelope,envelope_params,joints,n)

In [15]:
#check that finite differences and theoretical gradients are the same

#(l_hat(s,2,'exponential',[1.5],joints) - l_hat(s,2,'exponential',[1.499],joints)) / 0.001
#gradients_l_hat(s,2,'exponential',[1.5],joints)


#(l_hat(s,2,'gamma',[1.7,2.001],joints) - l_hat(s,2,'gamma',[1.7,2],joints)) / 0.001
#(l_hat(s,2,'gamma',[1.7001,2],joints) - l_hat(s,2,'gamma',[1.7,2],joints)) / 0.0001
#gradients_l_hat(s,int(2),'gamma',(1.7,2.),joints)
#gradients_l_hat(s,k,envelope,(1.2,),joints)

In [27]:
np.array(0,1)

TypeError: Cannot interpret '1' as a data type

# TESTS 

In [37]:
########## test the jax gamma implementation ############
values_to_use = np.load('values_to_use.npy')
nr_trawls = len(values_to_use)
J = np.array((0,1)) 
joints = [values_to_use[i+J] for i in range(nr_trawls - max(J))]
mu_L, V_L = -4.5, 3.5**2
envelope = 'gamma'
envelope_params = (1.5,0.75)
s =1
print(np.exp(composite_likelihoods(joints,J,mu_L,V_L,Sigma_tilde_func(s,J,envelope,envelope_params))),'\n')
print(np.sum(composite_likelihoods(joints,J,mu_L,V_L,Sigma_tilde_func(s,J,envelope,envelope_params))))

r = composite_likelihoods(joints,J,mu_L,V_L,Sigma_tilde_func(s,J,envelope,envelope_params))


delta_x = 10**(-6)
r_1 = composite_likelihoods(joints,J,mu_L+delta_x,V_L,Sigma_tilde_func(s,J,envelope,envelope_params))
V_L_delta_x = (3.5 + delta_x)**2
r_2 = composite_likelihoods(joints,J,mu_L,V_L_delta_x,Sigma_tilde_func(s,J,envelope,envelope_params))
r_3 = composite_likelihoods(joints,J,mu_L,V_L,Sigma_tilde_func(s,J,envelope,tuple(np.array(envelope_params)+delta_x*np.array([1,0]))))
r_4 = composite_likelihoods(joints,J,mu_L,V_L,Sigma_tilde_func(s,J,envelope,tuple(np.array(envelope_params)+delta_x*np.array([0,1]))))


[0.01069418 0.01144373 0.00109475 0.00130094 0.00641031 0.00548688
 0.01137941 0.01128808 0.00677853 0.00377304 0.01052604 0.01328496
 0.00837449 0.00825044 0.01323407 0.00733058 0.00758415 0.00660776
 0.00869977 0.00975882 0.00986842 0.01278278 0.01346302 0.0131613
 0.01049424 0.01110491 0.01239933 0.01114926 0.00384613 0.00385762
 0.01123756 0.01059477 0.01091636 0.01118781 0.01010795 0.00602821
 0.0072709  0.00569537 0.0054869  0.00887055 0.00806918 0.00344959
 0.00526264 0.01014372 0.01170979 0.00243662 0.00292541 0.00044906
 0.00036973 0.00442589 0.00472201 0.01020964 0.00632621 0.00448139
 0.00696812 0.01283453 0.01011578 0.0099234  0.01226767 0.00527511
 0.00433136 0.00556596 0.00406644 0.00957763 0.01048162 0.01174543
 0.0129471  0.00223479 0.00235471 0.01177148 0.01191816 0.0132249
 0.01254844 0.01271211 0.01269782 0.00954817 0.00938845 0.00545113
 0.00527369 0.01349995 0.00705862 0.00244723 0.00707908 0.01123404
 0.0115444  0.00467269 0.00480663 0.01242002 0.01224389 0.008884

In [38]:
r_all = np.array([r,(r_1-r)/delta_x, (r_2 - r)/ delta_x, (r_3 - r)/ delta_x, (r_4 - r)/delta_x])
np.save(arr=r_all,file ='r_all.npy')

In [17]:
def define_trawl_function(envelope,envelope_params):
    assert envelope in ['exponential','gamma','ig']
    assert isinstance(envelope_params,tuple) and isinstance(jump_part_params,tuple)

    if envelope == 'exponential':
    
        assert len(envelope_params) == 1
        lambda_  = envelope_params[0]
        trawl_function = lambda x :  lambda_ * np.exp(x * lambda_) * (x<=0)
        
    elif envelope == 'gamma':
        
        assert len(envelope_params) == 2
        H,delta = envelope_params
        trawl_function = lambda x :  H/delta * (1-x/delta)**(-H-1) * (x<=0)
        
    elif envelope == 'ig':
        
        assert len(envelope_params) == 2
        gamma,delta = envelope_params
        #total_area = gamma/delta change of varialbe ()**-0.5 = z
        trawl_function = lambda x : (delta/gamma) * (1-2*x/gamma**2)**(-0.5) *np.exp(delta * gamma * (1- (1-2*x/gamma**2)**0.5)) * (x<=0)
        
    return trawl_function

#np.random.seed(1)

nr_simulations   = 2
nr_trawls = 200
tau = 0.35
decorrelation_time = -np.inf
gaussian_part_params = (1.75,2.5)
jump_part_name   = None 
jump_part_params = (0,0)
#envelope parameters
envelope =  'exponential'    #envelope  is one of  ['exponential','gamma','ig']
TRUE_ENVELOPE_PARAMS  = (1.,)
trawl_function = define_trawl_function(envelope,TRUE_ENVELOPE_PARAMS)
J = np.array((0,1))
#np.random.seed(0)
trawl_slice = trawl(nr_trawls = nr_trawls, nr_simulations = nr_simulations,
                    trawl_function = trawl_function, tau =  tau,
               decorrelation_time =  decorrelation_time, gaussian_part_params = gaussian_part_params,
               jump_part_name = jump_part_name ,jump_part_params = jump_part_params)   

trawl_slice.simulate('slice','diagonals')
simulation_nr_to_use=0
values = trawl_slice.values[simulation_nr_to_use] 
joints = [values[i+J] for i in range(nr_trawls - max(J))]


In [18]:
optimise_l_hat(tau,J,envelope,(0.5,),joints,nr_trawls)

2.309789267861551 7.425597879461895
2.3097892678615506 5.009235286358456
2.309789267861551 5.724976421201918
2.3097892678615515 5.755317231597498
2.309789267861551 5.796083674982654
2.309789267861551 5.793904471295346
2.309789267861551 5.7939658779112335


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 835.2947759496554
        x: [ 8.588e-01]
      nit: 5
      jac: [ 2.530e-06]
     nfev: 7
     njev: 7
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>

In [19]:
100000* (l_hat(tau,J,envelope,(0.82,),joints,nr_trawls) - l_hat(tau,J,envelope,(0.81999,),joints,nr_trawls))

2.309789267861552 5.894744840820119
2.3097892678615515 5.894772258121317


2.726907769101672

In [20]:
gradients_l_hat(tau,J,envelope,(0.8214884,),joints,nr_trawls)

2.6146578999566543

In [21]:
mu_L_V_L(tau,J,envelope,(2.5,),joints,nr_trawls)

(2.3097892678615515, 4.848205112204911)

In [22]:
lags_to_use 

NameError: name 'lags_to_use' is not defined

In [ ]:
#remove rows with zeros in u_CL
indeces = [data_frames_list[i].u_CL == 0 for i in range(len(data_frames_list))]
index_1 = indeces[0].copy()
for i in range(1,len(indeces)):
    index_1 = np.logical_or(index_1,indeces[i])

print(f'we exclude {sum(index_1)}/{nr_simulations} simulations')

processed_data_frames_list = [i[~index_1] for i in data_frames_list]
true_df = pd.DataFrame(data = [(envelope_params_true + gaussian_part_params_true)*2],columns=['u_GMM','mu_GMM','scale_GMM','u_CL','mu_CL','scale_CL'])
true_df_extended = pd.DataFrame(data = [(envelope_params_true + gaussian_part_params_true)*2],columns=['u_GMM','mu_GMM','scale_GMM','u_CL','mu_CL','scale_CL'],index = k_list)

######  TO DO: ACF PLOTS FOR BAD CASES ###############
###### CHECK u = 0.1 #########################

In [ ]:
#median_error = pd.concat([np.median(processed_data_frames_list[i] - true_df_extended,axis=0)  for i in range(len(processed_data_frames_list))])
#edian_error.index = list(range(len(processed_data_frames_list)))
data_frames_list[0]



In [ ]:
#pd.concatenate([np.median(processed_data_frames_list[i]- true_df_extended,axis=0)  for i in range(len(processed_data_frames_list))])#,\
             #columns=['u_GMM','mu_GMM','scale_GMM','u_CL','mu_CL','scale_CL'])#,index=k_list)

In [ ]:
pd.DataFrame([np.mean(np.square((processed_data_frames_list[i]- true_df.values)),axis=0)  for i in range(len(processed_data_frames_list))],\
             columns =['u_GMM','mu_GMM','scale_GMM','u_CL','mu_CL','scale_CL'],index = k_list)

#GMM problems with higher lags because we didn t divide by the abs value of the correlation probably (not confident)
#mu_CL might increase after 10 lags due to problems with approximating \Sigma inverse with small nr of trawls on a delta grid
#scale_CL keeps decreasing even for lags >20 and shows convergence


### do histograms

In [ ]:
###### print(np.median(np.square(df_exp.u_GMM -u)))
#print(np.median(np.square(df_exp.u_CL -u)))
#data_frames_list[12][120:150]
import pickle

with open('data_frames_list_pickled.pkl', 'wb') as f:
      pickle.dump(data_frames_list, f)

In [ ]:
#print(np.median(np.abs(df_exp.mu_GMM -gaussian_part_params1[0])))
#print(np.median(np.abs(df_exp.mu_CL -gaussian_part_params1[0])))
with open('data_frames_list_pickled.pkl', 'rb') as f:
       mynewlist = pickle.load(f)

In [ ]:
#print(np.median(np.abs(df_exp.scale_GMM -gaussian_part_params1[1])))
#print(np.median(np.abs(df_exp.scale_CL -gaussian_part_params1[1])))

In [ ]:
#df_exp.isnull().any().any()
[pd.DataFrame.equals(data_frames_list[i],mynewlist[i]) for i in range(19)]

#### The loss function looks monotonic for exponential trawls

In [ ]:
#dsk
#
values_exp = trawl_exponential_1.result[-3]
k_exp=2
joints_exp = [np.array(values_exp[i:i+k_exp]) for i in range(0,len(values_exp) - k_exp +1 )]

x = np.linspace(max(u-2,0.1),u+2,100)
z = [l_hat(s,k_exp,'exponential',[i],joints_exp) for i in x]
plt.plot(x,z,label='loss')
plt.axvline(x=u,color = 'r',label='u')
plt.legend()

In [ ]:
values_gamma = trawl_gamma_1.result[-3]
k_gamma=10
joints_gamma = [np.array(values[i:i+k_gamma]) for i in range(0,len(values_gamma) - k_gamma +1 )]

x=np.linspace(max(0.1,H-0.4),H+0.4,75)
y = np.linspace(max(0.1,delta-0.6), delta + 0.6, 75)
X,Y = np.meshgrid(x, y)

Z = np.array([[l_hat(s,k_gamma,'gamma',[X[i][j],Y[i][j]],joints_gamma) for i in range(len(X))] for j in range(len(Y))])

In [ ]:
plt.contourf(X, Y, Z, cmap='RdGy')
plt.colorbar();

In [ ]:
l_hat(0.1,10,'exponential',(1.25,),((1,2),(2,3),(3,4)))

In [ ]:
#np.log(np.linalg.det(Sigma_tilde_func(0.1,30,'exponential',(1.25,))))

In [ ]:
np.log((1-np.exp(-2 * 1.25 * 0.1))**(30-1))
k=10
values = trawl.result[-3]
joints = [np.array(values[i:i+k]) for i in range(0,len(values) - k +1 )]
envelope = 'exponential'
envelope_params = (1.25,)
s=0.1


In [ ]:
l_hat(s,k,envelope,envelope_params,joints)

In [ ]:
gradients_l_hat(0.5,10,envelope,envelope_params,joints)

In [ ]:
envelope

In [ ]:
np.linalg.det(Sigma_tilde_func(tau,np.array([0,1,2,3]),envelope,(2.25,)))

In [ ]:
J

In [ ]:
np.exp(-20*0.1)

In [ ]:
l1